In [174]:
# Import librairies

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [175]:
# Display csv to dataframe
df = pd.read_csv(r'C:\Users\33760\Desktop\FULLSTACK\FULLSTACK\Projet\BLOC 3\Walmart\data\clean_file_walmart.csv')
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
3,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
4,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [176]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = df.drop(target_variable, axis = 1)
Y = df.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
2    1244390.03
3    1644470.66
4    1857533.70
Name: Weekly_Sales, dtype: float64

X :


,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,11.0,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
3,6.0,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
4,4.0,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [177]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =0.2, random_state = 1)

print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [178]:
# Save the original column names
feature_names = X_train.columns.tolist()

In [179]:
# Separate numeric/categorical columns
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Weekday']
categorical_features = ['Store', 'Holiday_Flag']

In [180]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown ='ignore', drop = 'first'))
])


In [181]:
preprocessor = ColumnTransformer(transformers = [
    ('cat', categorical_transformer, categorical_features),
    ('num', numeric_transformer, numeric_features)
])

In [182]:
print("Preprocessing X_train...")
print(X_train.head())
print()
X_train = preprocessor.fit_transform(X_train)
print("...Done!")
print(X_train[0:5,:]) # X_train is now a numpy array
print() 

Preprocessing X_train...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
56   10.0           NaN        43.43       3.287  127.191774         8.744   
94    7.0           0.0        39.30       3.936  197.722738         8.090   
35   16.0           0.0          NaN       3.659  198.126718         6.061   
38    1.0           1.0        38.51       2.548  211.242170         8.106   
93   11.0           1.0        69.90       2.735  215.061403         7.564   

      Year  Month   Day  Weekday  
56     NaN    NaN   NaN      NaN  
94  2012.0    4.0  20.0      4.0  
35  2012.0    9.0  14.0      4.0  
38  2010.0    2.0  12.0      4.0  
93  2010.0   11.0  26.0      4.0  

...Done!
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.         -0.94097806 -0.12627711 -1.39438569  1.34393812  0.12456822
  -0.05720272 

In [183]:
# Test pipeline
print("Preprocessing X_test...")
print(X_test.head())
print()
X_test = preprocessor.transform(X_test)
print("...Done!")
print(X_test[0:5,:]) # X_test is now a numpy array
print() 

Preprocessing X_test...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
69    15.0           0.0        79.97       3.972  135.873839         7.806   
46    14.0           NaN        41.76       3.625  184.994368         8.549   
58     3.0           0.0        45.71       2.572  214.424881         7.368   
114    3.0           0.0        73.44       3.594  226.968844         6.034   
73    20.0           0.0        39.79       3.739  213.472512         6.961   

       Year  Month   Day  Weekday  
69      NaN    NaN   NaN      NaN  
46   2011.0    3.0  25.0      4.0  
58   2010.0    2.0   5.0      4.0  
114  2012.0   10.0  19.0      4.0  
73      NaN    NaN   NaN      NaN  

...Done!
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.          0.          0.          0.          0.
   0.          1.19560335  1.40222057 -1.16944728  0.40067606  0.12456822
  -

In [184]:
# Train model

print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("...Done.")

Train model...
...Done.


In [185]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[1811324.505708    482480.14228712  587861.84376682 1573955.21318016
 1671994.595484   1967331.24105506  399877.3662813  1386669.73197778
 1907850.07551515 1610780.23130492  525145.80946968 2207944.27909626
 1988108.77883165 1564735.89522569 1179700.53935501 1336341.19168454
  556367.29617002 1594888.61592527 1519718.29979726 1850763.19514304
  285293.22686707 2017249.32451669 1938581.20228187  219206.61186549
  351234.0229148  1753118.94423583  935873.61886848  286720.47959628
  994967.85974339 1939264.85260038  586150.7755027  1955475.65794551
 1688367.24155444 1134605.38476036 1482608.1193329  1521926.55405719
 2078150.56522726 2056700.38610087 2119954.89806582 2136552.24250577
 1595118.43188802  405686.41931357  481572.0514169   361066.26042401
  379055.64462111  909249.61550878 1444810.0726976  1914632.56731192
 1534453.94968912 1063030.56774689  371998.26992221  454235.88856018
 1867347.33850173 2057091.85834575  902259.63161426 2127801.167

In [186]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[ 599801.24257857 2039576.78600081  422832.96405758  468229.28575301
 1981485.09050173  638817.02391246 1032649.33293176 1832441.87678847
 1922805.3097631   979926.57337369  418737.94407188  173850.57176535
 1582423.51378937 1899147.43870571  466329.70768858 2227954.07134507
 1994982.4552802  1450971.56654474  526727.91447067 2051854.82875551
 2113683.2271641   598090.56060765 2030552.91424899 1888016.19587168]



In [187]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(y_test, Y_test_pred))

R2 score on training set :  0.9733501248908372
R2 score on test set :  0.954365742398211


### It seems that our model  a bit. R2 train score is higher than R2 test score. Our model seems unable to generalise its performance over the data set on which it is not trained. Even though it seems to remain relatively balanced

In [188]:
# Get features weights 
weights = regressor.coef_
weights

array([ 3.62044429e+05, -1.20625287e+06,  6.63852319e+05, -1.33725530e+06,
        1.83458404e+04, -9.74439230e+05, -7.02377149e+05, -1.19413125e+06,
        4.75607960e+05,  3.64870552e+04,  5.93728259e+05,  6.55561870e+05,
       -7.11338175e+05, -1.03221799e+06, -5.61463682e+05, -2.47749398e+05,
        6.49924688e+04,  3.89559280e+05,  8.57428844e+02, -1.93726070e+04,
       -2.06057973e+04,  1.00141893e+05, -2.73351172e+04, -3.56244870e+03,
        5.15659496e+04, -3.98565405e+04,  0.00000000e+00])

In [189]:
# Associate each weight with its corresponding column
weights_dict = dict(zip(feature_names, weights))
weights_dict

{'Store': 362044.4290331271,
 'Holiday_Flag': -1206252.8733116998,
 'Temperature': 663852.318892876,
 'Fuel_Price': -1337255.2994587,
 'CPI': 18345.840383437346,
 'Unemployment': -974439.2299637842,
 'Year': -702377.1488266144,
 'Month': -1194131.247591501,
 'Day': 475607.9601536676,
 'Weekday': 36487.055193280976}

In [190]:
# Associate each weight with its corresponding column
sorted_weights = sorted(weights_dict.items(), key=lambda x: x[1], reverse=True)

# Show values ​​sorted in descending order
for item in sorted_weights:
    print(item[0], ": ", item[1])

Temperature :  663852.318892876
Day :  475607.9601536676
Store :  362044.4290331271
Weekday :  36487.055193280976
CPI :  18345.840383437346
Year :  -702377.1488266144
Unemployment :  -974439.2299637842
Month :  -1194131.247591501
Holiday_Flag :  -1206252.8733116998
Fuel_Price :  -1337255.2994587


## Ridge regularization & Griedsearch

In [191]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.09, 0.1, 0.12, 0.13] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 6 ) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.12}
Best R2 score :  0.8980058453720576


In [194]:
# Print R^2 scores with griedsearch 
print("R2 score on training set : ", gridsearch.score(X_train, y_train))
print("R2 score on test set : ", gridsearch.score(X_test, y_test))

R2 score on training set :  0.9709126184859799
R2 score on test set :  0.9416177859388987


In [193]:
# Print R^2 scores without griedsearch 
print("R2 score on training set : ", r2_score(y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(y_test, Y_test_pred))

R2 score on training set :  0.9733501248908372
R2 score on test set :  0.954365742398211


#### Here, we can see that the model's generalized performance hasn't improved by using a Ridge regularization (tuning the value of the regularization strength) and gried serach Howewer the model remain balanced.
